In [1]:
from TopoAnalysis import dem as d

In [12]:
elevation = d.Elevation.load('rapel_dem_utm.tif')
print(elevation._georef_info.xllcenter, elevation._georef_info.yllcenter)

176634.6565922048 6098100.209305331


In [11]:
import os

print(f"Current working directory: {os.getcwd()}")
print(f"Is file accessible? {os.path.isfile('rapel_dem_utm.tif')}")

Current working directory: /Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver
Is file accessible? True


In [10]:
# Set the working directory to where your file is located
os.chdir('/Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver')

# Verify the working directory
print(f"New working directory: {os.getcwd()}")

New working directory: /Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver


In [13]:
filled = d.FilledElevation(elevation = elevation)

In [14]:
flow_direction = d.FlowDirectionD8(flooded_dem = filled)
area = d.Area(flow_direction = flow_direction)
log_area = d.LogArea(area = area)

In [15]:
filled.save('rapel_dem_utm_filled')
flow_direction.save('rapel_fd_utm')
area.save('rapel_area_utm')
log_area.save('rapel_logarea_utm')

In [19]:
## Convert Lat long to easting northing
from pyproj import Transformer

def lat_long_to_utm_epsg32719(lat, lon):
    """
    Converts latitude and longitude to UTM easting and northing using EPSG: . . ..

    Parameters:
        lat (float): Latitude in decimal degrees.
        lon (float): Longitude in decimal degrees.

    Returns:
        tuple: (easting, northing) in meters.
    """
    # Define the transformer for EPSG:4326 (WGS 84) to EPSG:32719 (WGS 84 / UTM Zone 19S)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32719", always_xy=True)
    # Transform coordinates
    easting, northing = transformer.transform(lon, lat)
    return easting, northing

# Coords
coordinates = [
    (-71.7458024, -33.9458637),
     (-71.70305556, -34.05527778),
    (-71.70333333, -33.96972222),
     (-71.5784708, -34.1531149)
]

# Perform conversion and print results
for lon, lat in coordinates:
    easting, northing = lat_long_to_utm_epsg32719(lat, lon)
    print(f"{easting:.2f}, {northing:.2f}")



246233.51, 6240449.46
250505.71, 6228417.76
250229.41, 6237907.31
262282.00, 6217862.57


In [22]:
samples = [['RP-S2', 246233.51, 6240449.46],
           ['RP-S3', 250637.710, 6228479.760],
           ['RP-S3u', 250129.41, 6237907.31],
           ['RP-S4u', 262282.00, 6217862.57]
           ]


In [23]:
for sample, x, y in samples:
    try:
        basin_mask = d.Mask(flow_direction = flow_direction, outlets = ((x, y),))
        basin_mask.save(sample.replace('-','_') + '_mask')

        print(f"Finished processing sample: {sample}")

    except Exception as e: 
        print(f"failed to process sample {sample}: {e}")

Finished processing sample: RP-S2
Finished processing sample: RP-S3
Finished processing sample: RP-S3u
Finished processing sample: RP-S4u


In [24]:
import numpy as np

for sample, _, _ in samples:
    basin_mask = d.Mask.load(sample.replace('-','_') + '_mask')
    total_area = np.sum(basin_mask._griddata)*np.power(basin_mask._georef_info.dx,2)
    print(sample, float(total_area) / 1.0E6, "km2")

RP-S2 34.10789855061399 km2
RP-S3 113.89535734808891 km2
RP-S3u 209.9260827876405 km2
RP-S4u 9.432051824068205 km2


In [25]:
import os
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# List of masked raster files (replace with your actual file paths)
masked_rasters = ["RP_S2_mask", "RP_S3_mask", "RP_S3u_mask", "RP_S4u_mask"]  # Add your files here

# Output folder for polygon shapefiles
output_folder = "output_polygons"
os.makedirs(output_folder, exist_ok=True)

# Specify the CRS (optional). Example: "EPSG:32719" for UTM Zone 19S
user_defined_crs = "EPSG:32719"  # Change or set to None to use the raster's CRS

# Loop through each masked raster and convert to polygon
for raster_file in masked_rasters:
    try:
        # Open the raster file
        with rasterio.open(raster_file) as src:
            # Read the raster data
            image = src.read(1)  # Read the first band
            mask = image > 0  # Only include non-zero pixels
            
            # Convert raster to polygons
            results = (
                {"properties": {"value": value}, "geometry": shape(geom)}
                for geom, value in shapes(image, mask=mask, transform=src.transform)
            )
            
            # Convert to a GeoDataFrame
            gdf = gpd.GeoDataFrame.from_features(results, crs=src.crs)
            
            # Override CRS if user_defined_crs is provided
            if user_defined_crs:
                gdf = gdf.set_crs(user_defined_crs, allow_override=True)
                print(f"CRS set to: {user_defined_crs}")
            
            # Save as a shapefile
            output_file = os.path.join(output_folder, os.path.basename(raster_file).replace(".tif", "_polygon.shp"))
            gdf.to_file(output_file)
            print(f"Successfully converted {raster_file} to {output_file}")
    
    except Exception as e:
        print(f"Failed to process {raster_file}: {e}")


CRS set to: EPSG:32719
Successfully converted RP_S2_mask to output_polygons/RP_S2_mask
CRS set to: EPSG:32719
Successfully converted RP_S3_mask to output_polygons/RP_S3_mask
CRS set to: EPSG:32719
Successfully converted RP_S3u_mask to output_polygons/RP_S3u_mask
CRS set to: EPSG:32719
Successfully converted RP_S4u_mask to output_polygons/RP_S4u_mask
